In [ ]:
import numpy as np
import random
import time
import heapq

class GameWorld:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.world = np.full((width, height), 'T')  # Initialize with 'T' for terrain
        self.player_pos = (0, 0)  # Start player at top-left corner
        self.update_player_position()

    def update_obstacles_and_resources(self, obstacle_count=15, resource_count=10):
        self.world = np.full((self.width, self.height), 'T')  # Reset to terrain
        positions = [(x, y) for x in range(self.width) for y in range(self.height) if (x, y) != self.player_pos]
        random.shuffle(positions)

        # Place new obstacles and resources
        for _ in range(obstacle_count + resource_count):
            if positions:
                x, y = positions.pop()
                self.world[x, y] = 'O' if _ < obstacle_count else 'R'

        self.update_player_position()  # Ensure player position is updated in the world

    def update_player_position(self):
        x, y = self.player_pos
        self.world[x, y] = 'P'

    def print_grid(self):
        for row in self.world:
            print(' '.join(row))
        print("\n")

    def heuristic(self, current, goal):
        return abs(current[0] - goal[0]) + abs(current[1] - goal[1])  # Manhattan distance heuristic

    def astar(self, start, goal):
        frontier = []
        heapq.heappush(frontier, (0, start))
        came_from = {}
        cost_so_far = {}
        came_from[start] = None
        cost_so_far[start] = 0

        while frontier:
            _, current = heapq.heappop(frontier)

            if current == goal:
                break

            for next_node in self.neighbors(current):
                new_cost = cost_so_far[current] + 1  # Assuming uniform cost for each step
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    priority = new_cost + self.heuristic(goal, next_node)
                    heapq.heappush(frontier, (priority, next_node))
                    came_from[next_node] = current

        path = []
        current = goal
        while current != start:
            path.append(current)
            current = came_from[current]
        path.append(start)
        path.reverse()
        return path

    def neighbors(self, current):
        x, y = current
        candidates = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
        return [(nx, ny) for nx, ny in candidates if 0 <= nx < self.width and 0 <= ny < self.height and self.world[nx, ny] != 'O']

    def bfs_explore(self, start, visited, path):
        frontier = [start]
        while frontier:
            current = frontier.pop(0)
            if current not in visited:
                visited.add(current)
                path.append(current)
                neighbors = self.neighbors(current)
                for neighbor in neighbors:
                    frontier.append(neighbor)

    def dfs_explore(self, current, visited, path):
        if current not in visited:
            visited.add(current)
            path.append(current)
            neighbors = self.neighbors(current)
            for neighbor in neighbors:
                self.dfs_explore(neighbor, visited, path)

    def hill_climbing(self):
        current_pos = self.player_pos
        while True:
            neighbors = self.neighbors(current_pos)
            max_neighbor = max(neighbors, key=lambda pos: self.world[pos[0], pos[1]])
            if self.world[max_neighbor[0], max_neighbor[1]] > self.world[current_pos[0], current_pos[1]]:
                current_pos = max_neighbor
                self.player_pos = current_pos
            else:
                break

def print_menu():
    print("Select an objective:")
    print("1. Resource Collection")
    print("2. Hill Climbing")
    print("3. Exit")
    print("4. Exploration")
    choice = input("Enter choice (1-4): ")
    return int(choice)

def main():
    game_world = GameWorld(10, 10)
    while True:
        game_world.update_obstacles_and_resources()
        game_world.print_grid()

        choice = print_menu()
        if choice == 1:
            start = game_world.player_pos
            goals = [(x, y) for x in range(game_world.width) for y in range(game_world.height) if game_world.world[x, y] == 'R']
            if not goals:
                print("No resources found.")
                continue
            for goal in goals:
                path = game_world.astar(start, goal)
                print("Collecting resources along path:", path)
                time.sleep(1)  # Simulating resource collection time
                game_world.world[goal] = 'T'  # Mark resource as collected
                game_world.player_pos = goal  # Update player position to collected resource
                game_world.print_grid()  # Print updated grid after collecting resource
            print("All resources collected.")
            pass
        elif choice == 2:
            print("Performing hill climbing to find the highest point...")
            game_world.hill_climbing()
            print("Highest point found at:", game_world.player_pos)

        elif choice == 3:
            print("Exiting...")
            break
        elif choice == 4:
            print("Exploration:")
            print("1. BFS (Neighbor Exploration)")
            print("2. DFS (Deep Exploration)")
            exploration_choice = int(input("Enter choice (1-2): "))
            if exploration_choice == 1:
                start = game_world.player_pos
                visited = set()
                path = []
                game_world.bfs_explore(start, visited, path)
                print("BFS exploration complete.")
                print("Path taken:", path)
            elif exploration_choice == 2:
                start = game_world.player_pos
                visited = set()
                path = []
                game_world.dfs_explore(start, visited, path)
                print("DFS exploration complete.")
                print("Path taken:", path)
        else:
            print("Invalid choice. Please enter a number between 1 and 4.")

        time.sleep(2)  # Wait for 2 seconds before the next update

if __name__ == "__main__":
    main()


P T T T T T T T T T
T T T T T O T T T T
T O T T T R R T T T
T T T T T R O O O T
R R T T T O T T O T
T T T O T T R T O T
R R T R O T T T T T
T T T O T T T O T T
T T R T T T T O T T
T T T T T T O T O T


Select an objective:
1. Resource Collection
2. Hill Climbing
3. Exit
4. Exploration
Enter choice (1-4): 1
P T T T T T T T T T
T T T T T O T T T T
T O T T T T R T T T
T T T T T R O O O T
R R T T T O T T O T
T T T O T T R T O T
R R T R O T T T T T
T T T O T T T O T T
T T R T T T T O T T
T T T T T T O T O T


P T T T T T T T T T
T T T T T O T T T T
T O T T T T T T T T
T T T T T R O O O T
R R T T T O T T O T
T T T O T T R T O T
R R T R O T T T T T
T T T O T T T O T T
T T R T T T T O T T
T T T T T T O T O T


P T T T T T T T T T
T T T T T O T T T T
T O T T T T T T T T
T T T T T T O O O T
R R T T T O T T O T
T T T O T T R T O T
R R T R O T T T T T
T T T O T T T O T T
T T R T T T T O T T
T T T T T T O T O T


P T T T T T T T T T
T T T T T O T T T T
T O T T T T T T T T
T T T T T T O O O T
T R T 

KeyboardInterrupt: Interrupted by user